In [1]:
import autograd.numpy as np
import autograd.scipy as sp

import autograd

import sys
sys.path.insert(0, './../../LinearResponseVariationalBayes.py')
sys.path.insert(0, './../src/vb_modeling/')

import LinearResponseVariationalBayes as vb
import LinearResponseVariationalBayes.ExponentialFamilies as ef

from numpy.polynomial.hermite import hermgauss

import gmm_clustering_lib as gmm_utils
from shift_only_lib import LinearSensitivity
import common_modeling_lib 
 

from scipy import spatial
import scipy.cluster.hierarchy as sch

import matplotlib.pyplot as plt
%matplotlib inline

import time
from tqdm import tqdm

from copy import deepcopy

from matplotlib.patches import Ellipse
import matplotlib.cm as cm

np.random.seed(453453)



In [2]:
k_approx = 12
n_samples = 3

# Testing function to get weights from sticks in a way that is autograd differentiable

In [3]:
stick_lengths = np.random.beta(1, 4, size = (n_samples, k_approx - 1))

In [4]:
weights = common_modeling_lib.get_mixture_weights_array(stick_lengths)

In [5]:
weights1 = np.zeros((n_samples, k_approx))
weights2 = np.zeros((n_samples, k_approx))
for k in range(k_approx): 
    weights1[:, k] = common_modeling_lib.get_kth_weight_from_sticks(stick_lengths, k)
    
for n in range(n_samples): 
    weights2[n, :] = common_modeling_lib.get_mixture_weights(stick_lengths[n, :])

In [6]:
assert np.max(np.abs(weights - weights1)) < 1e-14

In [7]:
assert np.max(np.abs(weights - weights2)) < 1e-14

# Function that gets expected number of clusters from stick lengths

In [8]:
vb_params = gmm_utils.get_vb_params(dim = 3, k_approx = k_approx, n_obs = 150, gh_deg = 8)

vb_global_free_params = np.random.random(len(vb_params['global'].get_free()))
vb_params['global'].set_free(vb_global_free_params)

In [10]:
common_modeling_lib.get_e_number_clusters_from_logit_sticks_diffble(vb_params)

4.743216209849701

In [11]:
# make sure that is differentiable wrt to global_vb_params

In [12]:
def f(vb_params, vb_global_free_params): 
    vb_params['global'].set_free(vb_global_free_params)
    return common_modeling_lib.get_e_number_clusters_from_logit_sticks_diffble(vb_params)

In [13]:
df = autograd.grad(f, argnum=1)
df(vb_params, vb_global_free_params)

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -5.77154503e-01, -6.04112391e-01, -5.51977116e-01, -3.45259021e-01,
       -8.34017743e-02, -3.09957929e-02, -1.51070170e-02, -1.35778038e-03,
        1.69660724e-03,  2.28287234e-03,  2.15605124e-03,  3.51215610e-02,
        5.23452446e-02,  2.17795088e-02,  1.85680202e-02,  2.65595327e-02,
        9.46476022e-03,  